In [2]:
import numpy as np 
import pandas as pd 

In [3]:
data = pd.read_csv("Data_without-invalid_cells.csv")

In [4]:
data.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,ID,tweet_id,text,missing_text,Text_Only_Informative,Image_Only_Informative,Directed_Hate,Generalized_Hate,Sarcasm,Allegation,Justification,Refutation,Support,Oppose
0,0,0,0,0,1.052240e+18,New post Domestic Violence Awareness Hasnt Cau...,0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,1,1,1,1,1.052210e+18,Domestic Violence Awareness Hasn’t Caught Up W...,0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2,2,2,2,1.052180e+18,Mother Nature’s \n,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,4,4,4,1.052100e+18,It is high time named and shamed men in media...,0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,5,5,5,5,1.052050e+18,“Theres this idea that persistence involves sy...,0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [5]:
import tensorflow as tf
import sklearn
import seaborn as sbs
import sklearn.naive_bayes 
import sklearn.model_selection
import sklearn.metrics

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [6]:
!pip3 install bert-for-tf2

In [18]:
import tensorflow_hub as hub 
from bert import bert_tokenization
module_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2'
bert_layer = hub.KerasLayer(module_url, trainable=True)

In [19]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = bert_tokenization.FullTokenizer(vocab_file, do_lower_case)

In [20]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence) + [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

def build_model(bert_layer, max_len=512):
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    net = tf.keras.layers.Dense(64, activation='relu')(clf_output)
    net = tf.keras.layers.Dropout(0.2)(net)
    net = tf.keras.layers.Dense(32, activation='relu')(net)
    net = tf.keras.layers.Dropout(0.2)(net)
    out = tf.keras.layers.Dense(1, activation='sigmoid')(net)
    
    model = tf.keras.models.Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(tf.keras.optimizers.Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [21]:
max_len = 100
X_train, X_val, y_train, y_val = sklearn.model_selection.train_test_split(data.text, data['Sarcasm'], 
                                                                          test_size=0.2, random_state=0)
X_train = bert_encode(X_train, tokenizer, max_len=max_len)
X_val = bert_encode(X_val, tokenizer, max_len=max_len)

In [22]:
model = build_model(bert_layer, max_len=max_len)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 100)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 100)]        0                                            
__________________________________________________________________________________________________
keras_layer_4 (KerasLayer)      [(None, 768), (None, 109482241   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

In [29]:
%%time
checkpoint = tf.keras.callbacks.ModelCheckpoint('model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)

train_history = model.fit(
    X_train, y_train, 
   
    epochs=10,
    callbacks=[checkpoint, earlystopping],
    batch_size=1,
    verbose=1
)

Epoch 1/10
5447/5447 [==============================] - ETA: 0s - loss: 0.0613 - accuracy: 0.9853WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


5447/5447 [==============================] - 204s 37ms/step - loss: 0.0613 - accuracy: 0.9853
Epoch 2/10
5446/5447 [============================>.] - ETA: 0s - loss: 0.0463 - accuracy: 0.9892WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


5447/5447 [==============================] - 204s 38ms/step - loss: 0.0463 - accuracy: 0.9892
Epoch 3/10
5447/5447 [==============================] - ETA: 0s - loss: 0.0384 - accuracy: 0.9905WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


5447/5447 [==============================] - 205s 38ms/step - loss: 0.0384 - accuracy: 0.9905
Epoch 4/10
5447/5447 [==============================] - ETA: 0s - loss: 0.0309 - accuracy: 0.9910WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


5447/5447 [==============================] - 205s 38ms/step - loss: 0.0309 - accuracy: 0.9910
Epoch 5/10
5446/5447 [============================>.] - ETA: 0s - loss: 0.0224 - accuracy: 0.9928WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


...

5447/5447 [==============================] - 204s 38ms/step - loss: 0.0229 - accuracy: 0.9930
Epoch 7/10
5447/5447 [==============================] - ETA: 0s - loss: 0.0201 - accuracy: 0.9939WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


5447/5447 [==============================] - 204s 38ms/step - loss: 0.0201 - accuracy: 0.9939
Epoch 8/10
5446/5447 [============================>.] - ETA: 0s - loss: 0.0199 - accuracy: 0.9928WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


5447/5447 [==============================] - 205s 38ms/step - loss: 0.0199 - accuracy: 0.9928
Epoch 9/10
5446/5447 [============================>.] - ETA: 0s - loss: 0.0166 - accuracy: 0.9954WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


5447/5447 [==============================] - 205s 38ms/step - loss: 0.0166 - accuracy: 0.9954
Epoch 10/10
5447/5447 [==============================] - ETA: 0s - loss: 0.0130 - accuracy: 0.9950WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


5447/5447 [==============================] - 204s 38ms/step - loss: 0.0130 - accuracy: 0.9950
CPU times: user 27min 41s, sys: 5min 37s, total: 33min 19s
Wall time: 34min 4s


In [30]:
test_performance=model.evaluate(X_val,y_val)
print("The loss of the model on the test set : {}".format(test_performance[0]))
print("The accuracy of the model on the test set : {}%".format(test_performance[1]*100))



43/43 [==============================] - 5s 121ms/step - loss: 0.2533 - accuracy: 0.9772
The loss of the model on the test set : 0.25329431891441345
The accuracy of the model on the test set : 97.72393703460693%


Validation accuracy:  0.9750367107195301
